# Анализ оценок вина

Установка нужных библиотек для машинного обучения:

In [320]:
import importlib.util
import subprocess
import sys

def install_if_missing(package):
    if importlib.util.find_spec(package) is None:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])

for pkg in ["torch", "pandas", "numpy","matplotlib"]:
    install_if_missing(pkg)

In [321]:
import csv
import numpy as np
import torch

In [322]:
wine_path = "data\wine.csv" # Путь к файлу с датасетом
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1) # Путь к бд, вид данных входных, разделить, пропустить первую строку т.к. там название столбцов
col_list = next(csv.reader(open(wine_path), delimiter=';'))
print(wineq_numpy, wineq_numpy.shape, wineq_numpy.dtype,'\n', '\n', col_list)


[[ 7.    0.27  0.36 ...  0.45  8.8   6.  ]
 [ 6.3   0.3   0.34 ...  0.49  9.5   6.  ]
 [ 8.1   0.28  0.4  ...  0.44 10.1   6.  ]
 ...
 [ 6.5   0.24  0.19 ...  0.46  9.4   6.  ]
 [ 5.5   0.29  0.3  ...  0.38 12.8   7.  ]
 [ 6.    0.21  0.38 ...  0.32 11.8   6.  ]] (4898, 12) float32 
 
 ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']


In [323]:
wineq = torch.from_numpy(wineq_numpy)
print(wineq.shape, wineq.dtype)

torch.Size([4898, 12]) torch.float32


In [324]:
data = wineq[:, :-1] # Выбираем все строки и все столбцы, кроме последнего (оценки)
target = wineq[:, -1].long() # Выбираем последний столбец и все строки . преобразование в целые числа
print(data, data.shape, '\n', '\n', target, target.shape)

tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
        [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
        [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
        ...,
        [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
        [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
        [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]) torch.Size([4898, 11]) 
 
 tensor([6, 6, 6,  ..., 6, 7, 6]) torch.Size([4898])


Используем быстрое кодирование оценок для того, так как нет расстояния между оценками как между обычными естественными числами, дабы не было никаких отношений между числами

In [325]:
target_onehot = torch.zeros(target.shape[0], 10) # Создает матрицу высотой target, добавляет ширину 10 для оценок
target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [326]:
print(target_onehot[0, :], " - Пример первой строчки")

tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])  - Пример первой строчки


In [327]:
target_unsqueezed = target.unsqueeze(1)
print(target_unsqueezed, target_unsqueezed.shape)

tensor([[6],
        [6],
        [6],
        ...,
        [6],
        [7],
        [6]]) torch.Size([4898, 1])


In [328]:
data_mean = torch.mean(data, dim=0) # dim=0 выполенение свертки по измерению 0
data_var = torch.var(data, dim=0)
print(data_mean.shape, data_var.shape)

torch.Size([11]) torch.Size([11])


$$
\text{data}_{\text{normalized}} = \frac{\text{data} - \text{data}_{\text{mean}}}{\sqrt{\text{data}_{\text{var}}}}
$$


In [329]:
data_normalized = (data - data_mean) / torch.sqrt(data_var)
print(data_normalized, '\n', '\n', data_normalized.shape)

tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
          1.3422e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]]) 
 
 torch.Size([4898, 11])


In [330]:
bad_indexes = target <= 3
print(bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum())
print("форма | тип значений | количество <= 3")

torch.Size([4898]) torch.bool tensor(20)
форма | тип значений | количество <= 3


In [331]:
bad_data = data[bad_indexes]
print(bad_data.shape)

torch.Size([20, 11])


In [332]:
bad_data = data[target <= 3]
mid_data = data[(target > 3) & (target < 7)]
good_data = data[target >= 7]

bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim=0)

for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


In [333]:
total_sulfur_threshold = 141.83
total_sulfur_data = data[:, 6]
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)

print(predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum())

torch.Size([4898]) torch.bool tensor(2727)


In [334]:
actual_indexes = target > 5

print(actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum())

torch.Size([4898]) torch.bool tensor(3258)


In [335]:
n_matches = torch.sum(actual_indexes & predicted_indexes).item()
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()

print("\n", "Правильно оценено вин: ", n_matches, "\n", 
      "вероятность что предсказания окажется правильным: ", n_matches / n_predicted, "\n",
        "Определили реально из хороших вин: ", n_matches / n_actual)


 Правильно оценено вин:  2018 
 вероятность что предсказания окажется правильным:  0.74000733406674 
 Определили реально из хороших вин:  0.6193984039287906
